In [227]:
import pandas as pd
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import warnings
warnings.simplefilter('ignore')

In [228]:
country_virus=pd.read_csv("country_by_date.csv")
country_age = pd.read_csv("age_ratio.csv")
country_age = country_age.rename(columns={'Entity': 'Country_Region'})
country_age = country_age.loc[(country_age['Year'] == 2017)]
country_age = country_age[['Country_Region','Age dependency ratio (% of working-age population) (% of working-age population)']]

In [229]:
country_virus

,Unnamed: 0,Country_Region,Date,ConfirmedCases
0,0,Afghanistan,2020-01-22,0.0
1,5456,Afghanistan,2020-02-12,0.0
2,5458,Afghanistan,2020-02-10,0.0
3,5459,Afghanistan,2020-02-09,0.0
4,5460,Afghanistan,2020-02-08,0.0
...,...,...,...,...
13315,5415,Zimbabwe,2020-03-05,0.0
13316,5416,Zimbabwe,2020-03-04,0.0
13317,5427,Zimbabwe,2020-02-15,0.0
13318,5429,Zimbabwe,2020-02-17,0.0


In [230]:
#create a new table to get the country demographics data
countries = country_virus['Country_Region'].unique()
countries = pd.DataFrame (countries)
countries = countries.rename(columns={0: 'Country_Region'})

new_table = pd.merge(countries, country_age, on='Country_Region', how='left')
new_table = new_table.rename(columns={'Age dependency ratio (% of working-age population) (% of working-age population)':'Age Dependency Ratio (% of working-age population)'})
new_table

,Country_Region,Age Dependency Ratio (% of working-age population)
0,Afghanistan,84.587278
1,Albania,44.059506
2,Algeria,55.028217
3,Andorra,NaN
4,Angola,96.898518
...,...,...
175,Venezuela,51.996675
176,Vietnam,43.288199
177,West Bank and Gaza,NaN
178,Zambia,89.613257


In [231]:
physician = pd.read_csv("physician_per_capita_edit.csv")
physician = physician[['Country Name','Most_Recent']]
physician = physician.rename(columns={'Country Name': 'Country_Region'})
physician = physician.rename(columns={'Most_Recent': 'Physician_per_thousand'})

In [232]:
other_demo = pd.read_csv("other_demo_info_edit.csv")

In [233]:
import datetime
ts = pd.Timestamp(year = 2011,  month = 11, day = 21,  
                  hour = 10, second = 49, tz = 'US/Pacific')
other_demo = other_demo[['country','hospibed','pop']]
#other_demo['quarantine time'] = other_demo.groupby(['Country_Region'])['quarantine'].max()
other_demo = other_demo.rename(columns={'country': 'Country_Region'})


In [234]:
other_demo

,Country_Region,hospibed,pop
0,Afghanistan,0.5,"38,928,346"
1,Albania,2.9,"2,877,797"
2,Algeria,1.9,"43,851,044"
3,Andorra,2.5,"77,265"
4,Antigua and Barbuda,3.8,"97,929"
...,...,...,...
175,Venezuela,0.8,"28,435,940"
176,Vietnam,2.6,"97,338,579"
177,Zambia,2.0,"18,383,955"
178,Zimbabwe,1.7,"14,862,924"


In [235]:

lockdown_dates=pd.read_csv("countryLockdowndates.csv")
lock_down =  lockdown_dates[['Country/Region','Date']]
lock_down = lock_down.rename(columns={'Country/Region': 'Country_Region','Date':'Lockdown Date'})

In [236]:
country_by_date=pd.read_csv("join-table/country_by_date.csv")
country_by_date
country_demographics_date = country_by_date.merge(country_age, on=['Country_Region'], how='left')
country_demographics_date = country_demographics_date.merge(physician, on=['Country_Region'], how='left')
country_demographics_date = country_demographics_date.merge(other_demo, on=['Country_Region'], how='left')
country_demographics_date = country_demographics_date.merge(lock_down, on=['Country_Region'], how='left')
country_demographics_date

,Unnamed: 0,i,Country_Region,Date,ConfirmedCases,Population Density (per sq. km),HDI,Testing By Date,Total Testing,Age dependency ratio (% of working-age population) (% of working-age population),Physician_per_thousand,hospibed,pop,Lockdown Date
0,0,0,Afghanistan,2020-01-22,0,56.937760,0.498,NaN,NaN,84.587278,0.3039,0.5,"38,928,346",24/03/2020
1,1,5476,Afghanistan,2020-01-23,0,56.937760,0.498,NaN,NaN,84.587278,0.3039,0.5,"38,928,346",24/03/2020
2,2,5475,Afghanistan,2020-01-24,0,56.937760,0.498,NaN,NaN,84.587278,0.3039,0.5,"38,928,346",24/03/2020
3,3,5474,Afghanistan,2020-01-25,0,56.937760,0.498,NaN,NaN,84.587278,0.3039,0.5,"38,928,346",24/03/2020
4,4,5473,Afghanistan,2020-01-26,0,56.937760,0.498,NaN,NaN,84.587278,0.3039,0.5,"38,928,346",24/03/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22565,13315,9279,Zimbabwe,2020-03-31,8,37.324591,0.535,NaN,NaN,78.656894,0.2020,1.7,"14,862,924",27/03/2020
22566,13316,9277,Zimbabwe,2020-04-01,8,37.324591,0.535,NaN,NaN,78.656894,0.2020,1.7,"14,862,924",27/03/2020
22567,13317,9372,Zimbabwe,2020-04-02,9,37.324591,0.535,NaN,NaN,78.656894,0.2020,1.7,"14,862,924",27/03/2020
22568,13318,9443,Zimbabwe,2020-04-03,9,37.324591,0.535,NaN,NaN,78.656894,0.2020,1.7,"14,862,924",27/03/2020


In [237]:
country_demographics_date.to_csv("demographics/country_demographics_by_date.csv")

In [238]:
country_demographics = country_demographics_date[['Country_Region','Population Density (per sq. km)','HDI',
                                                  'Total Testing','hospibed','pop','Age dependency ratio (% of working-age population) (% of working-age population)',
                                                 'Physician_per_thousand']]

In [239]:
country_demographics = country_demographics.drop_duplicates()

In [240]:
country_demographics.to_csv("decountry_demographics.csv")